# Limpieza de datos 
Realizado por : Rolando Ortiz Herbas

# 1 Cargar librerias de trabajo

In [52]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_selector

# 2 Cargar el dataset

In [53]:
ruta = "C:/Users/Roland/Documents/UNAD_MEXICO_Matematicas_2022/Semestre_2024_1/ZZZ_Word_y_PT/DatosClientes/Clientes_Dos_Mil.csv"
datasetT = pd.read_csv(ruta)

# 3 Muestra el dataset cargado

In [54]:
print(datasetT.shape)
datasetT.head()

(2000, 31)


,Nocliente,Ocupacion,NoPago,TotalMonto,EstadoContrato,Producto,Plazo,FechaInicioC,FechaAdjudicación,FechaUltimoPago,...,Genero,PersonaTipo,dtEstadoContrato,GrupoOrigenVenta,SemanasAdjud,Pagos9,Domicilia_Pago,Email,timetotV,Y
0,1,Ejecutivo (a),35,302783,Subastado con 013,Autos,60,2015-04-17,2015-04-17,2018-02-21,...,Mujer,Fisica,"Subasta (0,20]",evento,0.0,9,1,1,35,1
1,7601,Coordinador (a),38,211366,Subastado con 012,Autos,60,2015-02-20,2015-02-20,2018-02-08,...,Hombre,Fisica,"Subasta (0,20]",modulo-agencia,0.0,9,1,1,38,1
2,9697,Arquitecto (a),50,323640,Sorteo,Autos,60,2015-02-20,2016-01-15,2019-02-18,...,Mujer,Fisica,Sorteo,modulo-agencia,47.0,8,1,1,50,1
3,12999,Gestor (a),60,377036,Sorteo,Autos,60,2015-04-17,2015-07-17,2020-03-05,...,Mujer,Fisica,Sorteo,modulo-agencia,13.0,9,0,1,60,1
4,201301,Propietario (a),45,392349,Subastado con 013,Autos,60,2015-04-17,2015-04-17,2018-12-26,...,Hombre,Moral,"Subasta (0,20]",modulo-agencia,0.0,9,0,1,45,1


# 4 Ver las variables categóricas

In [55]:
# Con data.info() podemos ver las variables categóricas
# (Dtype=object)
datasetT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Nocliente           2000 non-null   int64  
 1   Ocupacion           2000 non-null   object 
 2   NoPago              2000 non-null   int64  
 3   TotalMonto          2000 non-null   int64  
 4   EstadoContrato      2000 non-null   object 
 5   Producto            2000 non-null   object 
 6   Plazo               2000 non-null   int64  
 7   FechaInicioC        2000 non-null   object 
 8   FechaAdjudicación   2000 non-null   object 
 9   FechaUltimoPago     2000 non-null   object 
 10  FechaProyectadaFin  2000 non-null   object 
 11  MontoVencido        2000 non-null   int64  
 12  Mensualidad         2000 non-null   int64  
 13  Ingresos            1958 non-null   float64
 14  Legal               520 non-null    object 
 15  EdadActual          2000 non-null   int64  
 16  FechaD

# 5 Ver los niveles de las variables categoricas

Para el proceso de limpieza y manipulación de datos, utilizaremos los métodos de Pandas:

isnull(): Comprueba si hay valores nulos en un DataFrame.

dropna(): Elimina filas o columnas con valores nulos de un DataFrame.

fillna(): Rellena los valores nulos con un valor específico.

astype(): Convierte el tipo de datos de una columna a un tipo de datos específico.

describe(): Muestra estadísticas descriptivas de un DataFrame.

corr(): Calcula la correlación entre columnas de un DataFrame.

plot(): Crea gráficos a partir de los datos de un DataFrame.

value_counts(): Cuenta los valores únicos en una columna de un DataFrame.

In [ ]:
# Identificar valores faltantes
display(datasetT.isnull().sum())

# Si hay valores faltantes, aplicar estrategias de completar valores o eliminación

In [ ]:
# datasetT.select_dtypes(include=['float64', 'int64']).describe()
desc = datasetT.describe()

# Formatear los números a 8 enteros y 2 decimales
desc_formatted = desc.applymap(lambda x: f"{x:10.2f}")

# Mostrar el DataFrame formateado
print(desc_formatted)

In [ ]:
df = pd.DataFrame(datasetT)

# Guardar el DataFrame en un archivo CSV

df.to_csv("C:/Users/Roland/Documents/UNAD_MEXICO_Matematicas_2022/Semestre_2024_1/ZZZ_Word_y_PT/DatosClientes/Clientes_Dos_Mil_Limpio.csv", index=False, quotechar='"', quoting=csv.QUOTE_NONNUMERIC)

# 5 Detectamos valores nulos para corregir el dataset

In [56]:
datasetT.isnull()
#Si lo prefieres puedes usar:
# datasetT.notnull()

,Nocliente,Ocupacion,NoPago,TotalMonto,EstadoContrato,Producto,Plazo,FechaInicioC,FechaAdjudicación,FechaUltimoPago,...,Genero,PersonaTipo,dtEstadoContrato,GrupoOrigenVenta,SemanasAdjud,Pagos9,Domicilia_Pago,Email,timetotV,Y
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1996,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1997,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1998,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
